In [ ]:
import numpy as np
import matplotlib.pyplot as plt

file_sinr = "/home/luca/Scrivania/TLCnet_prj/ns3-mmwave/output10/mcMmWaveSinrTime_5_10_12_01_2018_07_57_33.txt"
file_switch = "/home/luca/Scrivania/TLCnet_prj/ns3-mmwave/output10/mcMmWaveSwitchStats_5_10_12_01_2018_07_57_33.txt"

with open(file_sinr, 'r') as f:
    in_file_sinr = []
    for line in f:
        tokens = line.rstrip().split(' ')
        enb = int(tokens[2]) - 2
        # elimino le righe riferite a enb5
        if (enb != 3):
            in_file_sinr.append(line)
    f.close()

n_lines = len(in_file_sinr)
n_ues = 12;
n_mmwave_enbs = 3;
n_records = n_lines / (n_mmwave_enbs * n_ues);

times = np.zeros(n_records)
sinrs = np.zeros((n_ues, n_mmwave_enbs, n_records))

for i, line in enumerate(in_file_sinr):
    tokens = line.rstrip().split(' ')
    time = float(tokens[0])
    ue = int(tokens[1]) - 1
    enb = int(tokens[2]) - 2
    sinr = float(tokens[3])
    times[i / (n_mmwave_enbs * n_ues)] = time
    sinrs[ue, enb, i / (n_mmwave_enbs * n_ues)] = tokens[3]
    
with open(file_switch, 'r') as f: 
    in_file_switch = []
    for line in f:
        if line not in in_file_switch:
            tokens = line.rstrip().split(' ')
            enb = int(tokens[2]) - 2
            # elimino le righe riferite a enb5
            if (enb != 3):
                in_file_switch.append(line)
    f.close()

ues_switches = [[] for _ in range(n_ues)]

def find_index(vect, value):
    for i in range(len(vect)):
        if (vect[i] >= value):
            return i      
    return len(vect)

for line in in_file_switch:
    tokens = line.rstrip().split(' ')    
    time = float(tokens[1])
    ue = int(tokens[2]) - 1
    target_enb = int(tokens[3]) - 2
    ues_switches[ue].append((target_enb, find_index(times, time)))

for i in range(n_ues):
    fig = plt.figure(figsize = (20, 10))
    ax1 = plt.gca()
    ax1.set_xlim(0, 60)
    ax1.xaxis.set_ticks(np.arange(0, 60, 5))
    ax1.xaxis.set_ticklabels(np.arange(0, 60, 5))

    plt.title("SINR ue" + str(i + 1) + " / enbs")
    plt.xlabel("Time [s]")
    plt.ylabel('SINR [dB]')
    plt.grid(True)
    
    n_switches = len(ues_switches[i])
    switch_indices = np.zeros(n_switches, dtype = np.int32)
    connected_enb_sinr = np.zeros(n_records)
    first_start = ues_switches[i][0][1]
    for j, switch in enumerate(ues_switches[i]):
        target_enb, start = switch
        switch_indices[j] = start
        connected_enb_sinr[start:] = sinrs[i, target_enb, start:]
      
    plt.plot(times[first_start:], connected_enb_sinr[first_start:], lw = 8, color = 'k')
    plt.plot(times, sinrs[i, 0, :], '--', lw = 2, color = 'b')
    plt.plot(times, sinrs[i, 1, :], '--', lw = 2, color = 'r')
    plt.plot(times, sinrs[i, 2, :], '--', lw = 2, color = '#F9A602')
    plt.plot(times[switch_indices], connected_enb_sinr[switch_indices], 'o', lw = 4, color = '#00FF00')
    
    plt.legend(['connected enb', 'enb2', 'enb3', 'enb4', 'switch'])
    plt.show()